In [1]:
import numpy as np
import pandas as pd
import pandas_profiling
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline
import seaborn as sns

import plotly.graph_objects as go



In [6]:
df = pd.read_csv('/home/polina/Downloads/SupplyDemand_Difference_WeekdayHour.csv')

In [7]:
df.head()

,Weekday,Hour,SUM of Rides possible to serve - All Users on the app
0,Friday,0,-21.050000
1,Friday,1,49.700000
2,Friday,2,90.500000
3,Friday,3,-18.500000
4,Friday,4,-25.981982


In [57]:
X = df['SUM of Rides possible to serve - All Users on the app'].apply(lambda x: x*-1)
X = X.values.reshape(-1, 1)

In [58]:
from sklearn.preprocessing import QuantileTransformer
qt = QuantileTransformer(n_quantiles=10, random_state=0)
df['Normalized'] = qt.fit_transform(X) 

In [59]:
df_aggregated = df.groupby(by = ['Weekday', 'Hour']).aggregate('mean')
df_aggregated = df_aggregated.reset_index()

In [123]:
pre_pivoted  = df_aggregated[['Normalized', 'Weekday','Hour']]
#pivoted = pre_pivoted.pivot_table('WeekDay','Hour', 'Supply Demand score Squared hours')
pivoted = pre_pivoted.pivot_table(values = 'Normalized', index=['Weekday'],
                                columns =['Hour'], aggfunc=np.mean)

# columns = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
x_axis = [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
            17, 18, 19, 20, 21, 22, 23]
new_index= ['Sunday', 'Saturday', 'Friday', 'Thursday', 'Wednesday', 'Tuesday', 'Monday']
pivoted = pivoted.reindex(new_index)
# pivoted = pivoted[columns]
pivoted.to_csv('pivoted')

In [124]:
pivoted

Hour,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
Weekday,,,,,,,,,,,,,,,,,,,,,
Sunday,0.088982,0.300662,0.219009,0.227169,0.792383,0.289243,0.569463,0.222091,0.722626,0.645809,...,0.496112,0.171775,0.159907,0.105872,0.730810,0.212757,0.183036,0.087728,0.603143,0.777779
Saturday,0.597917,0.385443,0.723559,0.867366,0.889223,0.562918,0.385611,0.498016,0.687167,0.688285,...,0.322137,0.449855,0.444115,0.305167,0.057010,0.166973,0.074411,0.062937,0.000000,0.035083
Friday,0.777835,0.445876,0.257216,0.767934,0.783347,0.392463,0.559648,0.711649,0.900688,0.954892,...,0.845575,0.891017,0.871256,0.778432,0.954826,0.926646,0.825476,0.450125,0.025319,0.011614
Thursday,0.743408,0.153291,0.673262,0.777730,0.609934,0.353415,0.409427,0.571750,0.897634,0.877427,...,0.551978,0.308203,0.238840,0.895462,1.000000,0.944907,0.808079,0.224187,0.163330,0.708056
Wednesday,0.426144,0.450802,0.250210,0.685179,0.662987,0.517142,0.555980,0.672721,0.903566,0.888294,...,0.221371,0.095588,0.200823,0.903120,0.953919,0.860430,0.091248,0.033259,0.078775,0.126975
Tuesday,0.414901,0.602707,0.334014,0.372856,0.344169,0.338431,0.303606,0.735759,0.914441,0.932833,...,0.598398,0.337927,0.490275,0.276572,0.946978,0.899639,0.314919,0.069010,0.389767,0.322604
Monday,0.451750,0.505620,0.406290,0.588382,0.593215,0.550580,0.648355,0.834406,0.916211,0.927224,...,0.109959,0.184309,0.133922,0.114350,0.849767,0.790403,0.083382,0.056593,0.368909,0.164404


In [126]:
import plotly.graph_objects as go
import plotly.figure_factory as ff


fig = go.Figure(data=go.Heatmap(
                   z=pivoted,
                   x=x_axis,
                   y=list(pivoted.index),
    colorscale=[
        [0.0, "rgb(254, 254, 254)"],
        [0.1, "rgb(234, 250, 243)"],
        [0.2,  "rgb(214, 245, 231)"],
        [0.3, "rgb(154,232,195)"],   
        [0.4, "rgb(93,218,159)"],   
        [0.5, "rgb(53,209,135)"],        
        [0.6, "rgb(47,188,121)"],
        [0.7, "rgb(42,167,108)"],
        [0.8, "rgb(31,125,81)"],
        [0.9, "rgb(21,83,54)"],
        [1.0, "rgb(15,62,40)"]
    ]))


fig.update_layout(
    title=go.layout.Title(
#         text="Plot Title",
        xref="paper",
        x=0
    ),
    
    xaxis=go.layout.XAxis(
        title=go.layout.xaxis.Title(
            font=dict(
                family="Courier New, monospace",
                size=18,
                color="#7f7f7f"
            ) 
        ), type="category"
    ),
    yaxis=go.layout.YAxis(
        title=go.layout.yaxis.Title(
            font=dict(
                family="Courier New, monospace",
                size=18,
                color="#7f7f7f"
            )


        )
    )
)




fig.show()